## Text Modelling

In [1]:
# Find folders and text under a given path
import os

# For dataframe/tables
import pandas as pd

# NLTK package
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Gensim
from gensim import corpora
from gensim import models

# Visualization
import pyLDAvis
import pyLDAvis.gensim

C:\Users\acer\Anaconda3\envs\py35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# Text document folder
folder = 'speeches'

# Stop words set
ignorewords = set(stopwords.words("english"))

# Lemmatizer
lemmatizer = WordNetLemmatizer().lemmatize

#define dictionary  and corpus
dictionary = corpora.Dictionary()
corpus = list()

# Clean text
# 1. Word Tokenize text
# 2. Ignore stopwords and non alpha characters, words less than 4 in length
# 3. conver to lower case
# 4. Lemmatize the words
# 5. Update/Add to Dictionary
# 6. Create Bag of Words
def clean_corpus(text) :
    textlist = list()
    textlist.append([lemmatizer(word.lower()) for word in word_tokenize(text) \
                       if word not in ignorewords and word.isalpha() and len(word) > 4 ])
    dictionary.add_documents(textlist)
    for text in textlist :
        corpus.append(dictionary.doc2bow(text, allow_update=True))    
    return(None)



In [3]:
# Read, clean and convert the speeches in to Tf-idf
corpus_tfidf = dict(); corpus_dict = dict()
for fld in  os.listdir(folder) :
    corpus = [] # Re-Initialize
    dictionary = corpora.Dictionary() # Re-Initialize
    for file in os.listdir(folder + '\\' + fld) :
        txtfile = folder + '\\' + fld + '\\' + file
        with open(txtfile, 'r',errors='ignore') as fobj :        
            clean_corpus(fobj.read())       
    # Tf-idf model
    Tfidf = models.TfidfModel(corpus, smartirs='ntc')
    corpus_tfidf[fld] = Tfidf[corpus]
    corpus_dict[fld] = dictionary

In [4]:
# LSI model
lsi = dict()
num_topics = 5 
num_words = 15

for fld in corpus_tfidf.keys() :
    lsi[fld] = models.LsiModel(corpus_tfidf[fld], id2word=corpus_dict[fld], num_topics=num_topics)


In [5]:
# Display the topics and words associated
for fld in corpus_tfidf.keys() :    
    print('*'*80)
    print(fld.upper())
    print(pd.DataFrame({"Topic "+ str(i+1):[topic[0] for topic in lsi[fld].show_topic(i,num_words)] \
                                    for i in range(num_topics)}, index = [i+1 for i in range(num_words)] ))

********************************************************************************
ROMNEY
    Topic 1     Topic 2       Topic 3     Topic 4       Topic 5
1   liberal       found         reply       peace      applause
2     found   searching        cancel      regime       dreamer
3    crisis   requested         leave        ally         plant
4   program     perhaps        falcon     dreamer        saving
5    school     nothing         berta      saving      remember
6    budget       could       library      weapon        little
7     going     liberal        valley    applause  unidentified
8    little       going      provided     veteran      audience
9    dollar      crisis         tweet       plant         going
10   nation      budget          view     iranian     attacking
11  percent       reply  presidential       build    imaginable
12    build    strategy      applause        wage         meant
13  company      little        ronald  capability    unfairness
14   always  str

In [6]:
# LDA model
lda = dict()
num_topics = 5 
num_words = 15

for fld in corpus_tfidf.keys() :
    lda[fld] = models.LdaModel(corpus_tfidf[fld], id2word=corpus_dict[fld], num_topics=num_topics)


In [7]:
# Display the topics and words associated
for fld in corpus_tfidf.keys() :  
    print('*'*80)
    print(fld.upper())
    print(pd.DataFrame({"Topic "+ str(i+1):[topic[0] for topic in lda[fld].show_topic(i,num_words)] \
                                    for i in range(num_topics)}, index = [i+1 for i in range(num_words)] ))

********************************************************************************
ROMNEY
         Topic 1       Topic 2     Topic 3       Topic 4      Topic 5
1          reply         found       found       comment        kerry
2       applause        church   searching        closed      teacher
3         cancel        reject   requested       liberal       crisis
4        liberal     inherited     perhaps  conservatism        derek
5   unidentified       perhaps       berta         david       option
6          leave     searching      falcon    strengthen      forward
7          going       teacher     nothing        saving       school
8        library     requested   encourage         heart        voted
9         valley        father       could      tomorrow         want
10      provided        parent       tough       sounded       public
11      audience        wicked      budget       factory      percent
12      remember  unbelievable      growth         night     remember
13

In [8]:
# Visualize - LDA using pyLDAvis
vis = dict()
pyLDAvis.enable_notebook()
for fld in corpus_tfidf.keys() :
    vis[fld] = pyLDAvis.gensim.prepare(lda[fld],corpus=corpus_tfidf[fld],dictionary=corpus_dict[fld])

C:\Users\acer\Anaconda3\envs\py35\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


In [9]:
# Print topics in Romney's speeches
print(list(corpus_tfidf.keys())[0])
vis[list(corpus_tfidf.keys())[0]]

romney


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      58.332573        1       1 -0.004783  0.000715
2      25.448555        1       2  0.002106  0.003344
1       8.657436        1       3  0.001121 -0.001380
4       4.483789        1       4  0.000821 -0.001225
3       3.077641        1       5  0.000736 -0.001454, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
935   Default  0.000000         found  0.000000  30.0000  30.0000
3683  Default  0.000000     searching  0.000000  29.0000  29.0000
4526  Default  0.000000       comment  0.000000  28.0000  28.0000
5364  Default  0.000000     requested  0.000000  27.0000  27.0000
5790  Default  0.000000        closed  0.000000  26.0000  26.0000
2840  Default  0.000000       liberal  0.000000  25.0000  25.0000
5624  Default  0.000000       teacher  0.000000  24.0000  24.0000
91    Default  0.000000       perhaps  0.000000  23.0000  23.0000
5239  Default  0.000000         kerry  0.000000  22.0000  22.0000
126   Default  0.000000        crisis  0.000000  21.0000  21.0000
4331  Default  0.000000    strengthen  0.000000  20.0000  20.0000
1951  Default  0.000000  conservatism  0.000000  19.0000  19.0000
3688  Default  0.000000        school  0.000000  18.0000  18.0000
1307  Default  0.000000        always  0.000000  17.0000  17.0000
6318  Default  0.000000          want  0.000000  16.0000  16.0000
1758  Default  0.000000       student  0.000000  15.0000  15.0000
5387  Default  0.000000       percent  0.000000  14.0000  14.0000
4890  Default  0.000000         build  0.000000  13.0000  13.0000
5928  Default  0.000000         heart  0.000000  12.0000  12.0000
1282  Default  0.000000        strong  0.000000  11.0000  11.0000
1681  Default  0.000000          time  0.000000  10.0000  10.0000
3507  Default  0.000000        budget  0.000000   9.0000   9.0000
216   Default  0.000000        parent  0.000000   8.0000   8.0000
6218  Default  0.000000       forward  0.000000   7.0000   7.0000
6049  Default  0.000000        saving  0.000000   6.0000   6.0000
2420  Default  0.000000         david  0.000000   5.0000   5.0000
1255  Default  0.000000       program  0.000000   4.0000   4.0000
885   Default  0.000000        public  0.000000   3.0000   3.0000
514   Default  0.000000       poverty  0.000000   2.0000   2.0000
6173  Default  0.000000        choice  0.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
56     Topic5  0.006707      illegals  0.129518   0.5203  -8.4106
3505   Topic5  0.006838          stay  0.132386   0.5178  -8.3912
4403   Topic5  0.006954       kennedy  0.138755   0.4876  -8.3744
4233   Topic5  0.006446    tirelessly  0.131226   0.4676  -8.4502
5884   Topic5  0.008003      counting  0.167489   0.4399  -8.2340
4801   Topic5  0.008152       factory  0.181579   0.3775  -8.2155
2840   Topic5  0.012383       liberal  0.457597  -0.1286  -7.7974
62     Topic5  0.007637       telling  0.172494   0.3637  -8.2807
4331   Topic5  0.009449    strengthen  0.321676  -0.0466  -8.0678
6049   Topic5  0.008957        saving  0.287370   0.0126  -8.1213
998    Topic5  0.008559      tomorrow  0.276746   0.0049  -8.1668
2687   Topic5  0.007831        border  0.204603   0.2180  -8.2556
5928   Topic5  0.008568         heart  0.284819  -0.0228  -8.1657
4958   Topic5  0.007854          line  0.214258   0.1749  -8.2527
2863   Topic5  0.008065    everything  0.239710   0.0891  -8.2262
1913   Topic5  0.008078         night  0.242953   0.0773  -8.2245
3252   Topic5  0.007133   strengthens  0.156054   0.3956  -8.3489
1355   Topic5  0.008038        asking  0.281079  -0.0735  -8.2296
3235   Topic5  0.007840        enough  0.248262   0.0257  -8.2545
514    Topic5  0.007991       poverty  0.280727  -0.0780  -8.2354
910    Topic5  0.007113        vetoed  0.158740   0.3756  -8.3518
4890   Topic5  

In [10]:
# Print topics in Obama's speeches
print(list(corpus_tfidf.keys())[1])
vis[list(corpus_tfidf.keys())[1]]

obama


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      37.062366        1       1 -0.005314 -0.000669
0      24.586107        1       2  0.001220  0.004325
4      18.934675        1       3  0.003188 -0.002623
1      10.228410        1       4  0.000447 -0.000588
3       9.188454        1       5  0.000458 -0.000444, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
5261  Default  0.000000       teacher  0.000000  30.0000  30.0000
5153  Default  0.000000         iraqi  0.000000  29.0000  29.0000
3106  Default  0.000000      district  0.000000  28.0000  28.0000
3228  Default  0.000000  undocumented  0.000000  27.0000  27.0000
47    Default  0.000000         cable  0.000000  26.0000  26.0000
3322  Default  0.000000         cairo  0.000000  25.0000  25.0000
4843  Default  0.000000          fuel  0.000000  24.0000  24.0000
5179  Default  0.000000       company  0.000000  23.0000  23.0000
5033  Default  0.000000       station  0.000000  22.0000  22.0000
3160  Default  0.000000     stevenson  0.000000  21.0000  21.0000
131   Default  0.000000     character  0.000000  20.0000  20.0000
2124  Default  0.000000       student  0.000000  19.0000  19.0000
5778  Default  0.000000        weapon  0.000000  18.0000  18.0000
4265  Default  0.000000      mcdonald  0.000000  17.0000  17.0000
3932  Default  0.000000        coburn  0.000000  16.0000  16.0000
5497  Default  0.000000         phone  0.000000  15.0000  15.0000
684   Default  0.000000      internet  0.000000  14.0000  14.0000
6060  Default  0.000000         party  0.000000  13.0000  13.0000
901   Default  0.000000      facility  0.000000  12.0000  12.0000
2824  Default  0.000000    television  0.000000  11.0000  11.0000
6317  Default  0.000000        threat  0.000000  10.0000  10.0000
1148  Default  0.000000  hospitalized  0.000000   9.0000   9.0000
6540  Default  0.000000       amnesty  0.000000   8.0000   8.0000
3684  Default  0.000000   citizenship  0.000000   7.0000   7.0000
4875  Default  0.000000    discussion  0.000000   6.0000   6.0000
1560  Default  0.000000        medium  0.000000   5.0000   5.0000
396   Default  0.000000      standard  0.000000   4.0000   4.0000
2701  Default  0.000000          lane  0.000000   3.0000   3.0000
3446  Default  0.000000          tank  0.000000   2.0000   2.0000
7540  Default  0.000000      epilepsy  0.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
5730   Topic5  0.034153       injured  0.214132   0.5515  -8.2717
1998   Topic5  0.030615   cooperative  0.192465   0.5488  -8.3810
6079   Topic5  0.031006      protects  0.196734   0.5395  -8.3683
1622   Topic5  0.027006    disclosure  0.173892   0.5248  -8.5065
2425   Topic5  0.027075   programming  0.177016   0.5096  -8.5039
6143   Topic5  0.030773       element  0.201563   0.5078  -8.3759
4560   Topic5  0.031222       content  0.205664   0.5021  -8.3614
1088   Topic5  0.027197          cast  0.179367   0.5009  -8.4994
2124   Topic5  0.051587       student  0.341156   0.4982  -7.8593
1645   Topic5  0.033738   achievement  0.225189   0.4889  -8.2839
5778   Topic5  0.050465        weapon  0.357737   0.4287  -7.8812
6317   Topic5  0.043495        threat  0.346921   0.3108  -8.0299
2809   Topic5  0.042238       program  0.353941   0.2614  -8.0592
6434   Topic5  0.035463    innovation  0.263355   0.3822  -8.2340
1181   Topic5  0.044114        mccain  0.425798   0.1200  -8.0157
791    Topic5  0.036654        soviet  0.288711   0.3233  -8.2010
5153   Topic5  0.046537         iraqi  0.494051   0.0248  -7.9623
7008   Topic5  0.036949       soldier  0.309665   0.2613  -8.1930
3401   Topic5  0.035826         these  0.298339   0.2677  -8.2238
3209   Topic5  0.035071         learn  0.299753   0.2416  -8.2451
6947   Topic5  0.033896      training  0.271022   0.3083  -8.2792
5313   Topic5  